In [94]:
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt
import math
import json
import cv2
import os

import polyscope as ps
ps.init()

In [214]:
def read_cam_data(path="../gt_RayCasting/camera_data_temp.json"):
    cams = []
    with open(path) as f:
        cam_data = json.load(f)

    # Blender to O3D Coordinate Conversion Matrix - rotating -90 about x
    rot_mat = np.array([
    [1, 0, 0, 0],
    [0, 0, 1, 0],
    [0, -1, 0, 0],
    [0, 0, 0, 1]], dtype=np.float32)

    # 180 around y
    rot_y_mat = np.array([
        [-1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, -1, 0],
        [0, 0, 0, 1]
    ], dtype=np.float32)

    rot_x_mat = np.array([
        [1, 0, 0, 0],
        [0, -1, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]
    ], dtype=np.float32)

    rot_xm_mat = np.array([
        [1, 0, 0, 0],
        [0, -1, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]
    ], dtype=np.float32)

    rot_x90_mat = np.array([
        [0, 1, 0, 0],
        [-1, 0, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]
    ], dtype=np.float32)

    rot_xm90_mat = np.array([
        [0, -1, 0, 0],
        [1, 0, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]
    ], dtype=np.float32)

    for i in range(4):
        cam_info = cam_data[i]
        
        #Get Matricies
        # Extrinsics
        #ex_mat = np.linalg.inv(np.array(cam_info["extrinsic_mat"], dtype=np.float32)) # in world coordinate frame

        ex_mat = np.array(cam_info["extrinsic_mat"], dtype=np.float32)
        ex_cam_rot = rot_mat @ ex_mat # converting to o3d
        ex_cam_rot_y = ex_cam_rot @ rot_y_mat
        if i == 1:
            #ex_cam_rot = rot_mat @ ex_mat # converting to o3d
            ex_cam_rot_x = ex_cam_rot_y @ rot_x_mat

        #ex_world_rotated = np.linalg.inv(ex_cam_rot_y) #camera matrix - because thats what create_rays_pinhole_needs
            ex_world_rotated = ex_cam_rot_x
        elif i == 2:
            ex_cam_rot_y1 = ex_cam_rot_y @ rot_x90_mat
            ex_world_rotated = ex_cam_rot_y1
        elif i == 3:
            ex_cam_rot_x = ex_cam_rot_y @ rot_xm_mat
            ex_cam_rot_y1 = ex_cam_rot_x @ rot_xm90_mat
            ex_world_rotated = ex_cam_rot_y1
        else:
            ex_world_rotated = ex_cam_rot_y

        cam_pos = ex_world_rotated[:3, 3].tolist()
        print(f"cam_{i+1}: {cam_pos}")

        # # World→Camera from Blender
        # ex_mat = np.array(cam_info["extrinsic_mat"], dtype=np.float32)
        # # Invert → Camera→World
        # cam_to_world = np.linalg.inv(ex_mat)
        # # Convert to Open3D axes
        # ex_world_rotated = rot_mat @ cam_to_world @ np.linalg.inv(rot_mat)
        
        # Intrinsics
        in_mat = np.array(cam_info["intrinsics"]["intrinsic_matrix"], dtype=np.float32)

        cam_tmp = {
            'ex_tensor' : o3d.core.Tensor(ex_world_rotated),
            'in_tensor' : o3d.core.Tensor(in_mat),
            'w' : cam_info["intrinsics"]["width"],
            'h' : cam_info["intrinsics"]["height"]
        }
        cams.append(cam_tmp)
    return cams

In [215]:
cams_data = read_cam_data()
ps.remove_all_structures()

cam_1: [-0.9805809259414673, -5.295135498046875, 3.675057769214618e-07]
cam_2: [-0.9805809259414673, -5.295135498046875, 0.0]
cam_3: [-0.9805802702903748, -5.295135498046875, -5.554908966587391e-06]
cam_4: [-0.9805802702903748, -5.295135498046875, 5.554908966587391e-06]


In [217]:
def img_to_pcd(cams, dir='../renders_for_test'):
    
    pcds = []
    for i, cam in enumerate(cams):
        # #extrinsics = np.linalg.inv(cam['ex_tensor'].numpy()) # using cam local
        # extrinsics = cam['ex_tensor'].numpy() # using global
        # extrinsics = extrinsics.astype(np.float64)

        # cam_pos = extrinsics[:3, 3].tolist()
        # print(f"cam_{i+1}: {cam_pos}")

        # # if i == 1:
        # #     mat = np.array([[1,0,0, 10],
        # #             [0,1,0,0],
        # #            [0,0,1,0],
        # #            [0,0,0,1]], dtype=np.float32)
            
        # #     extrinsics = mat @ extrinsics
        # #     print(f"new cam 2: {extrinsics[:3, 3].tolist()}")

        intrinsics = cam['in_tensor'].numpy()
        intrinsics = intrinsics.astype(np.float64)

        cam_intrinsics = o3d.camera.PinholeCameraIntrinsic(
            cam['w'], cam['h'], intrinsics
        )

        depth_img_path = os.path.join(dir, f"renders/Camera_{i+1}/depth_png/depth_norm_0001.png")
        rgb_img_path = os.path.join(dir, f"renders/Camera_{i+1}/rgb/rgb_0001.png")
            
        #depth_img =load_normalized_depth(depth_img_path) # using opencv
        depth_img = o3d.io.read_image(depth_img_path)
        rgb_img = o3d.io.read_image(rgb_img_path)
        
        rgbd = o3d.geometry.RGBDImage.create_from_color_and_depth(rgb_img, depth_img, 1.0, 5.0, False) 
        pcd = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd, cam_intrinsics)
        
        pcds.append(pcd)
    
        # o3d.visualization.draw_geometries([frame, pcd])

    return pcds

img_pcds = img_to_pcd(cams_data)

downsampled = img_pcds[0].voxel_down_sample(voxel_size=0.01)
points = np.asarray(downsampled.points)
depth_pcd = ps.register_point_cloud("Cam_1 points", points)

transform = cams_data[0]['ex_tensor'].numpy()
depth_pcd.set_transform(transform)
print(f"point_cloud_transfrom 1: {depth_pcd.get_transform()}")



downsampled = img_pcds[1].voxel_down_sample(voxel_size=0.01)
points = np.asarray(downsampled.points)
depth_pcd2 = ps.register_point_cloud("Cam_2 points", points)

transform = cams_data[1]['ex_tensor'].numpy()
mat = np.array([[1,0,0,0],
        [0,1,0,0],
        [0,0,1,0],
        [0,0,0,1]], dtype=np.float32)

extrinsics = mat @ transform
print(f"new cam 2: {extrinsics[:3, 3].tolist()}")

depth_pcd2.set_transform(extrinsics)
print(f"point_cloud_transfrom 2: {depth_pcd2.get_transform()}")

# downsampled = img_pcds[2].voxel_down_sample(voxel_size=0.01)
# points = np.asarray(downsampled.points)
# depth_pcd = ps.register_point_cloud("Cam_3 points", points)

# transform = cams_data[2]['ex_tensor'].numpy()
# depth_pcd.set_transform(transform)
# print(f"point_cloud_transfrom 3: {depth_pcd.get_transform()}")

# downsampled = img_pcds[3].voxel_down_sample(voxel_size=0.01)
# points = np.asarray(downsampled.points)
# depth_pcd = ps.register_point_cloud("Cam_4 points", points)

# transform = cams_data[3]['ex_tensor'].numpy()
# depth_pcd.set_transform(transform)
# print(f"point_cloud_transfrom 4: {depth_pcd.get_transform()}")



ps.show()

ps.remove_all_structures()

# depth_pcds = []
# for i, pcd in enumerate(img_pcds):
#     downsampled = pcd.voxel_down_sample(voxel_size=0.01)
#     points = np.asarray(downsampled.points) #downsampled.points)
#     if points.size > 0:
#         depth_pcd = ps.register_point_cloud(f"Camera {i+1} points", points)
#         depth_pcds.append(depth_pcd)

# ps.show()
# for p in depth_pcds:
#     p.remove()

point_cloud_transfrom 1: [[ 1.9611609e-01  7.6370426e-08 -9.8058069e-01 -9.8058093e-01]
 [-9.8058069e-01  5.5261737e-08 -1.9611609e-01 -5.2951355e+00]
 [ 3.9211123e-08  1.0000000e+00  8.5725084e-08  3.6750578e-07]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00  1.0000000e+00]]
new cam 2: [-0.9805809259414673, -5.295135498046875, 0.0]
point_cloud_transfrom 2: [[-0.19611609  0.         -0.9805807  -0.9805809 ]
 [ 0.9805807   0.         -0.19611609 -5.2951355 ]
 [ 0.          1.          0.          0.        ]
 [ 0.          0.          0.          1.        ]]
